In [ ]:
import numpy as np
from pathlib import Path
from scipy.io import readsav
import h5py
import pickle
import cv2
# import diplib as dip
from IPython.display import clear_output
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from tqdm.notebook import tqdm

In [ ]:
def brightness_reconstruction(img): # doi: 10.1109/TPS.2018.2828863.
    im_norm = img / 255
    img = np.average(im_norm,axis=None)
    img = np.log(im_norm + 1) * (im_norm - img)
    img = img / np.max(img)
    img = np.where(img < 0, 0, img)
    return img * 255

def process_image(img, kernel_size, sigma, threshold, erode_kernel):
    # img = cv2.GaussianBlur(img,(kernel_size, kernel_size),0)
    img = brightness_reconstruction(img)
    # img = np.array(dip.MatchedFiltersLineDetector2D(img, sigma = sigma)) # 10.1109/42.34715
    img *= 255.0/img.max()
    img = brightness_reconstruction(img)
    img = np.where(img < threshold, 0, 1).astype('uint8')
    # img = cv2.erode(img, np.ones((erode_kernel,erode_kernel), np.uint8), iterations=1)
    return img

def canny(img):
    # gray=(255-255*(img-np.min(img))/(np.max(img)-np.min(img))).astype('uint8')

    # reduce the noise using Gaussian filters
    kernel_size = 11 
    blur_gray = cv2.GaussianBlur(img,(kernel_size, kernel_size),0)

    # Apply Canny edge detctor
    low_threshold = 10
    high_threshold = 20
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
    
    return edges

### Workflow

1. Loop across all files
1. Loop across all indicies in file
1. Get processed image, and r,l datapoints
1. Append to 3 arrays
1. After each full run, save process image array, r, l datapoint to hdf5

In [ ]:
tv_image_path = Path('tv_images')
inversion_data_path = Path('outputs/inversion_data')
hdf5_path = Path('outputs/hdf5')
files = sorted(tv_image_path.glob('*.sav'))
file_lengths = [len(readsav(str(file))['emission_structure'][0][3]) for file in files]
cumulative_lengths = np.insert(np.cumsum(file_lengths), 0, 0)
tv_dim = readsav(str(files[0]))['emission_structure'][0][7][0].shape
inversion_dim = readsav(str(files[0]))['emission_structure'][0][0][0].shape

In [ ]:
crop_dim = (240, 480)
kernel_size = 5
sigma = 1
threshold = 4
erode_kernel = 4
aspect_num = 1/2

### Raw and Points

In [ ]:
hdf5_file_name = hdf5_path / 'tv_raw.hdf5'
hf = h5py.File(hdf5_file_name, 'w') # open h5py file
tv_dataset = hf.create_dataset("tv_images", shape=(np.sum(file_lengths), tv_dim[0], tv_dim[1]), dtype='uint8')
points_dataset = hf.create_dataset("points", shape=(np.sum(file_lengths), 4), dtype='float32')

# Add datasets to the groups
for idx, file in enumerate(files):
    frames = readsav(file)['emission_structure'][0][3].astype(int)
    tv_image = readsav(file)['emission_structure'][0][7][frames]
    tv_image_process = np.asarray(tv_image) # faster process and convert to binary
    pkl_path = (inversion_data_path / file.stem).with_suffix('.pkl')
    with open(pkl_path, 'rb') as pkl_file:
            label_info = pickle.load(pkl_file)
    points = np.concatenate((label_info['l_location'], label_info['r_location']),1)
    
    for i in range(file_lengths[idx]):
        tv_dataset[cumulative_lengths[idx]+i] = tv_image_process[i]
        points_dataset[cumulative_lengths[idx]+i] = points[i]
hf.close()

### Process and Points

In [ ]:
hdf5_file_name = hdf5_path / 'tv_process_simple.hdf5'
hf = h5py.File(hdf5_file_name, 'w') # open h5py file
tv_dataset = hf.create_dataset("tv_images", shape=(np.sum(file_lengths), crop_dim[0], crop_dim[1]), dtype='uint8')
points_dataset = hf.create_dataset("points", shape=(np.sum(file_lengths), 4), dtype='float32')

# Add datasets to the groups
for idx, file in enumerate(files):
    print(f"{idx+1} of {len(files)}")
    frames = readsav(file)['emission_structure'][0][3].astype(int)
    tv_image = readsav(file)['emission_structure'][0][7][frames]
    pkl_path = (inversion_data_path / file.stem).with_suffix('.pkl')
    with open(pkl_path, 'rb') as pkl_file:
            label_info = pickle.load(pkl_file)
    points = np.concatenate((label_info['l_location'], label_info['r_location']),1)
    
    for i in range(file_lengths[idx]):
        tv_dataset[cumulative_lengths[idx]+i] = np.asarray(process_image(tv_image[i, 0:240, 240:720],kernel_size, sigma, threshold, erode_kernel))
        points_dataset[cumulative_lengths[idx]+i] = points[i]
    clear_output()
hf.close()

### Inversion and Points

In [ ]:
hdf5_file_name = hdf5_path / 'compiled_inversion_no_image.hdf5'
hf = h5py.File(hdf5_file_name, 'w') # open h5py file
rz_dataset = hf.create_dataset("rz", shape=(np.sum(file_lengths), 4), dtype='float32')
intensity_dataset = hf.create_dataset("intensity", shape=(np.sum(file_lengths), 2), dtype='float32')

# Add datasets to the groups
for idx, file in enumerate(files):
    pkl_path = (inversion_data_path / file.stem).with_suffix('.pkl')
    with open(pkl_path, 'rb') as pkl_file:
            label_info = pickle.load(pkl_file)
    points = np.concatenate((label_info['l_location'], label_info['r_location']),1)
    points_i = np.concatenate((label_info['l_intensity'], label_info['r_intensity']))

    for i in range(file_lengths[idx]):
            rz_dataset[cumulative_lengths[idx]+i] = points[i]
            intensity_dataset[cumulative_lengths[idx]+i] = points_i[i]
hf.close()

### Raw and Synthetic and Points

In [ ]:
hdf5_file_name = hdf5_path / 'x_outer_radiation.hdf5'
hf = h5py.File(hdf5_file_name, 'w') # open h5py file
tv_dataset = hf.create_dataset("tv_images", shape=(np.sum(file_lengths), tv_dim[0], tv_dim[1]), dtype='uint8')
points_dataset = hf.create_dataset("points", shape=(np.sum(file_lengths), 4), dtype='float32')
intensity_dataset = hf.create_dataset("intensity", shape=(np.sum(file_lengths), 2), dtype='float32')
print(files)
# Add datasets to the groups
for idx, file in enumerate(files):
    frames = readsav(file)['emission_structure'][0][3].astype(int)
    tv_image = readsav(file)['emission_structure'][0][7][frames]
    tv_image_process = np.asarray(tv_image) # faster process and convert to binary
    pkl_path = (inversion_data_path / file.stem).with_suffix('.pkl')
    with open(pkl_path, 'rb') as pkl_file:
            label_info = pickle.load(pkl_file)
    points = np.concatenate((label_info['x_location'], label_info['r_location']),1)
    points_i = np.concatenate((label_info['x_intensity'], label_info['r_intensity']))
    
    for i in range(file_lengths[idx]):
        tv_dataset[cumulative_lengths[idx]+i] = tv_image_process[i]
        points_dataset[cumulative_lengths[idx]+i] = points[i]
        intensity_dataset[cumulative_lengths[idx]+i] = points_i[i]
hf.close()

### Raw and Inversion and Points

In [ ]:
anti_file_lengths = [len(readsav(str(file))['emission_structure'][0][5]) - len(readsav(str(file))['emission_structure'][0][3]) for file in files]
anti_cum_len = np.insert(np.cumsum(anti_file_lengths), 0, 0)
tv_full_train = np.zeros((np.sum(file_lengths), 256, 256))
inversion_full_train = np.zeros((np.sum(file_lengths), 256, 256))
tv_full_test = np.zeros((np.sum(anti_file_lengths), 256, 256))

In [ ]:
for idx, file in enumerate(files):
    frames = readsav(file)['emission_structure'][0][3].astype(int)
    tv_image = readsav(file)['emission_structure'][0][7][frames]
    inversion = readsav(file)['emission_structure'][0][0]
    file_len = len(frames)
    resize_tv = np.flip(resize(tv_image, (file_len, 256, 256), order=0, preserve_range=True), axis=(2,1))
    resize_inversion = resize(inversion, (file_len, 256, 256), order=0, preserve_range=True)
    tv_full_train[cumulative_lengths[idx]:cumulative_lengths[idx+1]] = resize_tv / 127.5 - 1
    inversion_full_train[cumulative_lengths[idx]:cumulative_lengths[idx+1]] = resize_inversion / 127.5 - 1

In [ ]:
for idx, file in enumerate(files):
    frames = np.setdiff1d(readsav(file)['emission_structure'][0][5].astype(int), readsav(file)['emission_structure'][0][3].astype(int))
    tv_image = readsav(file)['emission_structure'][0][7][frames]
    file_len = len(frames)
    resize_tv = np.flip(resize(tv_image, (file_len, 256, 256), order=0, preserve_range=True), axis=(2,1))
    tv_full_test[anti_cum_len[idx]:anti_cum_len[idx+1]] = resize_tv / 127.5 - 1

In [ ]:
plt.imshow(tv_full_train[100])

In [ ]:
print(tv_full_train.shape)
print(inversion_full_train.shape)
print(tv_full_test.shape)

In [ ]:
out_path = Path('outputs')
with h5py.File(out_path / 'tv_inversion.h5', 'w') as f:
    f.create_dataset('A_train', data=tv_full_train)
    f.create_dataset('A_test', data=tv_full_test)
    f.create_dataset('B_train', data=inversion_full_train)

### TV Synthetic HDF5

In [ ]:
randomization = False

file_name_1 = Path('outputs/hdf5/s_outs_v3_limited.h5')
file_name_2 = Path('outputs/hdf5/x_outer_radiation.hdf5')

out_path = Path('outputs')

with h5py.File(file_name_1, 'r') as f:
    synthetic_images = f['image'][:] * 2 - 1
    
with h5py.File(file_name_2, 'r') as f:
    points = f['points'][:]
    tv_images = f['tv_images'][:] / 127.5 - 1

print(len(synthetic_images), len(tv_images))
file_len = 1840

crop_synthetic = resize(synthetic_images[:file_len], (file_len, 256, 256))
crop_tv = np.flip(resize(tv_images[:file_len], (file_len, 256, 256)), axis=1)


In [ ]:
print(synthetic_images.min(), synthetic_images.max())
print(tv_images.min(), tv_images.max())

In [ ]:
idx = 1799
axs1 = plt.subplot(1,2,1)
axs1.imshow(crop_synthetic[idx], origin='lower')
print(np.min(crop_synthetic[idx]), np.max(crop_synthetic[idx]))

axs2 = plt.subplot(1,2,2)
axs2.imshow(crop_tv[idx], origin='lower')
print(np.min(crop_tv[idx]), np.max(crop_tv[idx]))

plt.show()

In [ ]:
print(synth_train.shape)

In [ ]:
# tts_percent = 0.99
tvs_percent = 0.8

# synth_dat, synth_val, tv_dat, tv_val = train_test_split(crop_synthetic, crop_tv, train_size=tts_percent, random_state=42)
synth_train, synth_test, tv_train, tv_test = train_test_split(crop_synthetic, crop_tv, train_size=tvs_percent, random_state=42)

# print(len(synth_dat), len(synth_val), len(tv_dat), len(tv_val))
print(len(synth_train), len(synth_test), len(tv_train), len(tv_test))
with h5py.File(out_path / 'tv_synth.h5', 'w') as f:
    f.create_dataset('synth_train', data=synth_train)
    f.create_dataset('synth_test', data=synth_test)
    # f.create_dataset('synth_val', data=synth_val)
    f.create_dataset('tv_train', data=tv_train)
    f.create_dataset('tv_test', data=tv_test)
    # f.create_dataset('tv_val', data=tv_val)

In [ ]:
axs1 = plt.subplot(1,2,1)
axs1.imshow(crop_tv[211], origin='lower')
axs2 = plt.subplot(1,2,2)
axs2.imshow(crop_synthetic[10], origin='lower')
plt.show()

In [ ]:
file_name = 'outputs/tv_synth.h5'

with h5py.File(file_name, 'r') as f:
    synth_train = f['synth_train'][:]
    synth_test = f['synth_test'][:]
    # synth_val = f['synth_val'][:]
    tv_train = f['tv_train'][:]
    tv_test = f['tv_test'][:]
    # tv_val = f['tv_val'][:]


In [ ]:
idx = 884
axs1 = plt.subplot(1,2,1)
axs1.imshow(synth_train[idx], origin='lower')
# print(np.min(synth_val[idx]), np.max(synth_val[idx]))

axs2 = plt.subplot(1,2,2)
axs2.imshow(tv_train[idx], origin='lower')
# print(np.min(tv_val[idx]), np.max(tv_val[idx]))

plt.show()
